In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import os 
from tqdm import tqdm 
from copy import deepcopy
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
from shapely.geometry import Polygon, LineString, Point

file_path = './make_files'

-> 첫번째 파일 만들기

In [2]:
data_0 = gpd.read_file('files/grid/강동구/nlsp_021001001.shp', encoding='utf8')
print(data_0.head(10))
# del data_0

# 사용할 첫번째 데이터는 국토정보플랫폼 국토통계지도에서 가져온, 서울시 자치구 격자 데이터.
# 총인구, 총인구수전체(남녀통합), 격자, 100m 단위
# 크롤링이 아닌 다운받아야 하는 형태여서, 직접 가입하여 25개 자치구별로 files 폴더에 정리하였음.
# http://map.ngii.go.kr/ms/map/NlipMap.do?tabGb=statsMap
# lbl 컬럼은 격자 내 총인구 값의 str 형태, val 컬럼은 격자 내 총인구 값의 numpy.float 형태

        gid     lbl    val                                           geometry
0  다사682503  549.00  549.0  POLYGON ((968200.000 1950300.000, 968200.000 1...
1  다사708496   97.00   97.0  POLYGON ((970800.000 1949600.000, 970800.000 1...
2  다사710501  276.00  276.0  POLYGON ((971000.000 1950100.000, 971000.000 1...
3  다사677501  510.00  510.0  POLYGON ((967700.000 1950100.000, 967700.000 1...
4  다사689486  277.00  277.0  POLYGON ((968900.000 1948600.000, 968900.000 1...
5  다사667479   59.00   59.0  POLYGON ((966700.000 1947900.000, 966700.000 1...
6  다사696515   73.00   73.0  POLYGON ((969600.000 1951500.000, 969600.000 1...
7  다사682493  811.00  811.0  POLYGON ((968200.000 1949300.000, 968200.000 1...
8  다사671505  213.00  213.0  POLYGON ((967100.000 1950500.000, 967100.000 1...
9  다사676474  318.00  318.0  POLYGON ((967600.000 1947400.000, 967600.000 1...


In [3]:
data_0 = None
debug = False

# 모든 구에 대하여 전처리 및 병합 반복... + 자치구 라벨링
for name in tqdm(os.listdir('files/grid')):  
    data = gpd.read_file('files/grid/{}/nlsp_021001001.shp'.format(name), encoding='utf8')  
    data.drop('lbl', axis=1, inplace=True)  # lbl 컬럼은 사용하지 않음
    
    data = data[data['val'].notna()]    # 인구수가 0인 격자는 분석에서 제외함 (학교, 공원, 문화재 등....)
    data = data[data['val'] != 0].reset_index(drop=True)
    data['val'] = data['val'].astype(int)
    
#     data['center'] = data['geometry'].apply(lambda x: (x.bounds[0] + 50, x.bounds[1] + 50))  # center 컬럼 생성 : 격자 중심점
    
    data['gu'] = name  # 구 이름 컬럼 

    if isinstance(data_0, type(None)):      # 병합
        data_0 = data
    else:
        data_0 = pd.concat([data_0, data], axis=0)

data_0.reset_index(drop=True, inplace=True)
data_0.head(10)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:04<00:00,  6.16it/s]


,gid,val,geometry,gu
0,다사579454,136,"POLYGON ((957900.000 1945400.000, 957900.000 1...",강남구
1,다사585452,21,"POLYGON ((958500.000 1945200.000, 958500.000 1...",강남구
2,다사583456,440,"POLYGON ((958300.000 1945600.000, 958300.000 1...",강남구
3,다사597458,62,"POLYGON ((959700.000 1945800.000, 959700.000 1...",강남구
4,다사595438,24,"POLYGON ((959500.000 1943800.000, 959500.000 1...",강남구
5,다사638410,381,"POLYGON ((963800.000 1941000.000, 963800.000 1...",강남구
6,다사584464,23,"POLYGON ((958400.000 1946400.000, 958400.000 1...",강남구
7,다사587471,15,"POLYGON ((958700.000 1947100.000, 958700.000 1...",강남구
8,다사581459,321,"POLYGON ((958100.000 1945900.000, 958100.000 1...",강남구
9,다사592435,496,"POLYGON ((959200.000 1943500.000, 959200.000 1...",강남구


In [4]:
# 1. 국가공간정보포털 오픈마켓(http://data.nsdi.go.kr/dataset/20171206ds00001) : (센서스경계)행정동경계에서 Z_SOP_BND_ADM_DONG_PG.zip 다운로드 -> 경로에 압축 풀고, 파일 가져오기
# ( Z_SOP_BND_ADM_DONG_PG.shp )   
# 2. SGIS 통계지리정보서비스에서 SGIS 행정구역코드(https://sgis.kostat.go.kr/view/newhelp/dd_help_20_0) -> 로컬에서 가져와서, 1번 데이터에 행정동 이름 라벨링
# ( adm_code.xls )

local_path = './files/행정동/Z_SOP_BND_ADM_DONG_PG/Z_SOP_BND_ADM_DONG_PG.shp'
data_1 = gpd.read_file(local_path, encoding='utf8')
data_1 = data_1[data_1['ADM_DR_CD'].astype(int) < 2000000]   # ADM_DR_CD 컬럼은 행정동 코드. 서울 지역은 1로 시작하는 7자리 숫자이므로, 다음과 같이 전처리.
data_1.drop('ADM_DR_NM', axis=1, inplace=True)              # 분석에 의미 없는 열 삭제

local_path = './files/행정동/adm_code.xls'
data_2 =  pd.read_excel(local_path, sheet_name='2020년6월', header=0)
data_2 = data_2.rename(columns=data_2.iloc[0])
data_2 = data_2.drop(data_2.index[0])
data_2 = data_2[data_2['읍면동코드'].astype(int) < 2000000]      # 읍면동코드 컬럼은 행정동 코드. 서울 지역은 1로 시작하는 7자리 숫자이므로, 다음과 같이 전처리.

data_1 = data_1.to_crs({'init':'epsg:5179'})   # 좌표 체계 통일

data_1.reset_index(drop=True, inplace=True)
data_2.reset_index(drop=True, inplace=True)



C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [5]:
data_1.head(10)

,BASE_DATE,ADM_DR_CD,OBJECTID,geometry
0,20210630,1101053,1,"POLYGON ((953553.932 1953335.741, 953555.261 1..."
1,20210630,1101054,2,"POLYGON ((953844.081 1955492.177, 953858.694 1..."
2,20210630,1101055,3,"POLYGON ((952490.430 1956548.820, 952497.644 1..."
3,20210630,1101056,4,"POLYGON ((953683.878 1959209.871, 953665.233 1..."
4,20210630,1101057,5,"POLYGON ((952298.144 1953539.606, 952324.838 1..."
5,20210630,1101058,6,"POLYGON ((952572.047 1953258.829, 952573.125 1..."
6,20210630,1101060,7,"POLYGON ((954894.795 1954614.580, 954888.340 1..."
7,20210630,1101061,8,"POLYGON ((954918.389 1954371.538, 954926.411 1..."
8,20210630,1101063,9,"POLYGON ((956606.810 1953149.973, 956606.676 1..."
9,20210630,1101064,10,"POLYGON ((956365.890 1954112.187, 956371.710 1..."


In [6]:
data_2.head(10)

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭
0,11,서울특별시,11010,종로구,1101053,사직동
1,11,서울특별시,11010,종로구,1101054,삼청동
2,11,서울특별시,11010,종로구,1101055,부암동
3,11,서울특별시,11010,종로구,1101056,평창동
4,11,서울특별시,11010,종로구,1101057,무악동
5,11,서울특별시,11010,종로구,1101058,교남동
6,11,서울특별시,11010,종로구,1101060,가회동
7,11,서울특별시,11010,종로구,1101061,종로1·2·3·4가동
8,11,서울특별시,11010,종로구,1101063,종로5·6가동
9,11,서울특별시,11010,종로구,1101064,이화동


In [7]:
dict_ = {}
for i, j in zip(data_2['읍면동코드'], data_2['읍면동명칭']):
    dict_[str(i)] = j
data_1['ADM_DR_CD'] = [dict_[i] for i in data_1['ADM_DR_CD']]     # 딕셔너리를 통한 전처리

data_1.head(10)

,BASE_DATE,ADM_DR_CD,OBJECTID,geometry
0,20210630,사직동,1,"POLYGON ((953553.932 1953335.741, 953555.261 1..."
1,20210630,삼청동,2,"POLYGON ((953844.081 1955492.177, 953858.694 1..."
2,20210630,부암동,3,"POLYGON ((952490.430 1956548.820, 952497.644 1..."
3,20210630,평창동,4,"POLYGON ((953683.878 1959209.871, 953665.233 1..."
4,20210630,무악동,5,"POLYGON ((952298.144 1953539.606, 952324.838 1..."
5,20210630,교남동,6,"POLYGON ((952572.047 1953258.829, 952573.125 1..."
6,20210630,가회동,7,"POLYGON ((954894.795 1954614.580, 954888.340 1..."
7,20210630,종로1·2·3·4가동,8,"POLYGON ((954918.389 1954371.538, 954926.411 1..."
8,20210630,종로5·6가동,9,"POLYGON ((956606.810 1953149.973, 956606.676 1..."
9,20210630,이화동,10,"POLYGON ((956365.890 1954112.187, 956371.710 1..."


In [8]:
list_ = []
for i in tqdm(range(len(data_0))):
    cnt = False
    point = data_0['geometry'][i].bounds              # 격자의 각 꼭짓점 좌표
    point = Point([point[0] + 50, point[1] + 50])   # 격자 중심점의 좌표.. -> 격자를 그대로 사용하게 되면, 행정동이 겹치는 격자를 분류할 수가 없어서, 격자 중심점을 기준으로 분류
    for j in range(len(data_1)):
        boundary = data_1.loc[j, 'geometry']        # 행정동 경계의 Polygon 정보
        if boundary.contains(point):                # 격자 중심점이 행정동 경계 내에 포함되면:
            dong = data_1.loc[j, 'ADM_DR_CD']        
            list_.append(dong)                     # 리스트에 행정동 이름 append
            cnt = True
            break
    if not cnt:
        list_.append(None)                         # 격자 중심점이 서울 밖인 격자는 제외함
        
data_0['dong'] = list_
data_0 = data_0[[not isinstance(i, type(None)) for i in data_0['dong']]]   # 격자 중심점이 서울 밖인 격자는 제외함 : 12개의 격자 제외
data_0.reset_index(drop=True, inplace=True)

100%|███████████████████████████████████████████████████████████████████████████| 29516/29516 [01:06<00:00, 444.51it/s]


In [9]:
data_0.reset_index(drop=True, inplace=True)

In [10]:
local_path = os.path.join(file_path, 'data_0')   # 경로 설정
if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    pass
data_0.to_file(os.path.join(local_path, 'data_0.shp'), encoding='utf-8')   # 파일 저장 : ./make_files/data_0/data_0.shp

In [11]:
del data_0
del data_1
del data_2

-> 두번째 파일 만들기

In [12]:
# 문화체육관광부 -> 2021년 전국 공공체육시설 현황(https://www.mcst.go.kr/kor/s_policy/dept/deptView.jsp?pSeq=1571&pDataCD=0417000000&pType=)
# 다운로드 및 '생활체육관' sheet 읽기... -> 반다비 체육센터와 같은 형태인 '생활체육관'을 통한 분석이 가장 의미있을 것으로 판단

data_0 = pd.read_excel('files/붙임1전국공공체육시설(2020년말+기준,+총괄).xlsx', sheet_name='생활체육관', header=1).loc[4:108]
data_0 = data_0[['시군구', '시설명']]
data_0.reset_index(drop=True, inplace=True)

In [13]:
for idx, i in enumerate(data_0['시설명']):
    i = i.replace('사회복지관 체육관', '사회복지관').replace('청소년수련관', '청소년센터').replace('채육관', '체육관').replace(' 체육관', '').replace('\n', '').replace(' ', '').replace('청소년센터체육관', '청소년센터')
    data_0.loc[idx, '시설명'] = i
    
data_0.loc[3, '시설명'] = '충무스포츠센터'
data_0.loc[28, '시설명'] = '면목2동체육관'                 # 100개 남짓이므로, 위 로직으로 전처리하면 안되는 것들(정식명칭이 아닌 경우, 오타, 띄어쓰기 오류 등...)을 전처리
data_0.loc[18, '시설명'] = '마포 서울특별시여성능력개발원'   # selenium 으로 크롤링하는 것을 보고, 직접 라벨링
data_0.loc[31, '시설명'] = '성북종합레포츠타운'
data_0.loc[35, '시설명'] = '해오름스포츠센터'
data_0.loc[37, '시설명'] = '성북구민회관'
data_0.loc[42, '시설명'] = '중계구민체육센터'
data_0.loc[46, '시설명'] = '은평청소년수련관'
data_0.loc[49, '시설명'] = '마포아트센터'
data_0.loc[38, '시설명'] = '시립강북청소년센터'
data_0.loc[92, '시설명'] = '대진체육관'
data_0.loc[97, '시설명'] = '송파구체육문화회관'
data_0.loc[94, '시설명'] = '역삼청소년수련관'
data_0.loc[101, '시설명'] = '송파아우름체육센터'


In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By

driver = webdriver.Chrome("./chromedriver")
driver.get("https://map.kakao.com/")             # 카카오맵 크롤링
driver.implicitly_wait(3)

list_ = []
for name in data_0['시설명']:
    driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').send_keys('{}'.format(name))
    driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    try:
        word = driver.find_element_by_class_name('addr').find_element_by_tag_name('p').text
        list_.append(word)
    except:
        print(name)
        list_.append('None')
    driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').clear()
    time.sleep(.5)

data_0['주소'] = list_             # '주소' 열에 해당 시설 주소 저장

<ipython-input-14-e7847bc0185b>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")
<ipython-input-14-e7847bc0185b>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').send_keys('{}'.format(name))
<ipython-input-14-e7847bc0185b>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').send_keys(Keys.ENTER)
<ipython-input-14-e7847bc0185b>:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  word = driver.find_element_by_class_name('addr').find_element_by_tag_name('p').text
C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_* c

In [15]:
data_1 = data_0.copy()

for idx, i in enumerate(data_1['주소']):             # 주소를 필요한 부분만 선택하여 라벨링 -> 카카오맵 상에서 나온 주차장, 지하 등 정보 제외
    list_ = i.split()
    str_ = list_[0] + ' ' + list_[1] + ' ' + list_[2] + ' ' + list_[3]
    data_1.loc[idx, '주소'] = str_
    
data_1.loc[43, '주소'] = '서울 노원구 월계동 106-5'             # 카카오맵 상에서의 오타 처리..


In [16]:
data_1

,시군구,시설명,주소
0,종로구,올림픽기념국민생활관,서울 종로구 성균관로 91
1,종로구,종로구민회관,서울 종로구 지봉로5길 7
2,종로구,종로문화체육센터,서울 종로구 인왕산로1길 21
3,중구,충무스포츠센터,서울 중구 퇴계로 387
4,중구,손기정문화체육센터,서울 중구 손기정로 101-4
5,중구,중구회현체육센터,서울 중구 퇴계로12길 78
6,중구,장충문화체육센터,서울 중구 퇴계로50길 59
7,중구,남산타운문화체육센터,서울 중구 다산로 32
8,용산구,효창종합사회복지관,서울 용산구 효창원로 146-12
9,용산구,갈월종합사회복지관,서울 용산구 두텁바위로 25


In [17]:
# !pip install geopy
from geopy.geocoders import Nominatim                    # 주소를 통해 위도 / 경도 라벨링
geo_local = Nominatim(user_agent='South Korea')

def geocoding(address):                # 주소의 위도, 경도 반환하는 함수

    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]
    
latitude = []
longitude =[]
address = data_1['주소']

for i in tqdm(address):                    # 주소로부터 위도/경도를 저장
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])



100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [01:45<00:00,  1.00s/it]


In [18]:
import numpy as np
data_0[np.array(latitude) == 0]        # 위/경도 변환이 되지 않은 1개 시설(개운산스포츠센터)  

,시군구,시설명,주소
33,성북구,개운산스포츠센터,서울 성북구 북악산로 949-60


In [19]:
data_0['위도'] = latitude
data_0['경도'] = longitude
data_0.loc[33, '위도'] = 37.5967922            # 위/경도 변환이 되지 않은 1개 시설(개운산스포츠센터) -> 다울주소전환 (https://address.dawul.co.kr/) 을 통해 직접 라벨링
data_0.loc[33, '경도'] = 127.026362
data_0['geometry'] = data_0.apply(lambda row : Point([row['위도'], row['경도']]), axis=1)
data_0 = gpd.GeoDataFrame(data_0, geometry='geometry')
data_0.crs = ({'init':'epsg:4326'})


C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [20]:
dict_ = {'시군구' : 'gu', '시설명' : 'name', '주소' : 'Address', '위도' : 'lat', '경도' : 'long', '위/경도' : 'lat/long'} # geopandas dataframe 저장을 위한 컬럼명 변경
data_0.rename(columns=dict_, inplace=True)


In [21]:
local_path = os.path.join(file_path, 'data_1')   # 경로 설정
if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    pass
data_0.to_file(os.path.join(local_path, 'data_1.shp'), encoding='utf-8')   # 파일 저장 : ./make_files/data_1/data_1.shp